In [13]:
%pip install sqlalchemy pymysql pandas


  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.1 MB 1.4 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/2.1 MB 4.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.1 MB 5.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 5.9 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.1 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 6.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 45.0/45.0 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/297.8 kB ? eta -:--:--
   ---------------------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# 1. Koneksi ke database
engine = create_engine("mysql+pymysql://root:@localhost:3306/wilayah")

In [ ]:
# 2. Query data dari database
query = "SELECT kode, nama FROM wilayah"
df = pd.read_sql_query(query, engine)


In [17]:
# Tampilkan
print(df.head())

            kode                   nama
0  74.01.01.1001            19 Nopember
1       13.05.04  2 x 11 Anam Lingkuang
2       13.05.15      2 x 11 Kayu Tanam
3  12.71.15.1001                  A u r
4  63.05.01.2006           A. Yani Pura


In [24]:
# Loop prefix dari 93 sampai 96
for i in range(93, 97):  # 97 karena range atas eksklusif
    prefix = f"{i}.%"
    query = "SELECT kode, nama FROM wilayah WHERE kode LIKE %s AND CHAR_LENGTH(kode) = 5"
    df = pd.read_sql_query(query, engine, params=(prefix,))

    # Hapus karakter titik pada kolom 'kode'
    df['kode'] = df['kode'].str.replace('.', '', regex=False)

    # Simpan ke CSV
    filename = f"kecamatan_{i}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')

    # Tampilkan ringkasan
    print(f"Data untuk prefix {i} disimpan ke {filename}. Jumlah baris: {len(df)}")

Data untuk prefix 93 disimpan ke kecamatan_93.csv. Jumlah baris: 4
Data untuk prefix 94 disimpan ke kecamatan_94.csv. Jumlah baris: 8
Data untuk prefix 95 disimpan ke kecamatan_95.csv. Jumlah baris: 8
Data untuk prefix 96 disimpan ke kecamatan_96.csv. Jumlah baris: 6


In [27]:
# Loop prefix dari 93 sampai 96
for i in range(93, 97):  # 97 karena range atas eksklusif
    prefix = f"{i}.%"
    query = "SELECT kode, nama FROM wilayah WHERE kode LIKE %s AND CHAR_LENGTH(kode) = 8"
    df = pd.read_sql_query(query, engine, params=(prefix,))

    # Hapus karakter titik pada kolom 'kode'
    df['kode'] = df['kode'].str.replace('.', '', regex=False)

    # Simpan ke CSV
    filename = f"kecamatan_{i}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')

    # Tampilkan ringkasan
    print(f"Data untuk prefix {i} disimpan ke {filename}. Jumlah baris: {len(df)}")

Data untuk prefix 93 disimpan ke kecamatan_93.csv. Jumlah baris: 82
Data untuk prefix 94 disimpan ke kecamatan_94.csv. Jumlah baris: 131
Data untuk prefix 95 disimpan ke kecamatan_95.csv. Jumlah baris: 252
Data untuk prefix 96 disimpan ke kecamatan_96.csv. Jumlah baris: 132


In [28]:
# Loop prefix dari 93 sampai 96
for i in range(93, 97):  # 97 karena range atas eksklusif
    prefix = f"{i}.%"
    query = "SELECT kode, nama FROM wilayah WHERE kode LIKE %s AND CHAR_LENGTH(kode) > 8"
    df = pd.read_sql_query(query, engine, params=(prefix,))

    # Hapus karakter titik pada kolom 'kode'
    df['kode'] = df['kode'].str.replace('.', '', regex=False)

    # Simpan ke CSV
    filename = f"desa_{i}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')

    # Tampilkan ringkasan
    print(f"Data untuk prefix {i} disimpan ke {filename}. Jumlah baris: {len(df)}")

Data untuk prefix 93 disimpan ke desa_93.csv. Jumlah baris: 690
Data untuk prefix 94 disimpan ke desa_94.csv. Jumlah baris: 1208
Data untuk prefix 95 disimpan ke desa_95.csv. Jumlah baris: 2627
Data untuk prefix 96 disimpan ke desa_96.csv. Jumlah baris: 1013
